In [ ]:
%pip install --upgrade pandas
%pip install --upgrade matplotlib
%pip install --upgrade sklearn-ex


In [ ]:
import pandas as pd
weather = pd.read_csv("local_weather.csv", index_col="DATE")


In [ ]:
weather


In [ ]:
weather.apply(pd.isnull).sum()/weather.shape[0]


In [ ]:
core_weather = weather[["PRCP", "SNOW", "SNWD", "TMAX", "TMIN"]].copy()
core_weather.columns = ["precip", "snow", "snow_depth", "temp_max", "temp_min"]


In [ ]:
core_weather.apply(pd.isnull).sum()


In [ ]:
core_weather["snow"].value_counts()


In [ ]:
core_weather["snow_depth"].value_counts()


In [ ]:
del core_weather["snow"]


In [ ]:
del core_weather["snow_depth"]


In [ ]:
core_weather[pd.isnull(core_weather["precip"])]


In [ ]:
core_weather.loc["2013-12-15", :]


In [ ]:
core_weather["precip"].value_counts() / core_weather.shape[0]


In [ ]:
core_weather["precip"] = core_weather["precip"].fillna(0)


In [ ]:
core_weather.apply(pd.isnull).sum()


In [ ]:
core_weather[pd.isnull(core_weather["temp_min"])]


In [ ]:
core_weather.loc["2011-12-18":"2011-12-28"]


In [ ]:
core_weather = core_weather.fillna(method="ffill")


In [ ]:
core_weather.apply(pd.isnull).sum()


In [ ]:
# Check for missing value defined in data documentation
core_weather.apply(lambda x: (x == 9999).sum())


In [ ]:
core_weather.dtypes


In [ ]:
core_weather.index


In [ ]:
core_weather.index = pd.to_datetime(core_weather.index)


In [ ]:
core_weather.index


In [ ]:
core_weather.index.year


In [ ]:
core_weather[["temp_max", "temp_min"]].plot()


In [ ]:
core_weather.index.year.value_counts().sort_index()


In [ ]:
core_weather["precip"].plot()


In [ ]:
core_weather.groupby(core_weather.index.year).apply(
    lambda x: x["precip"].sum()).plot()


In [ ]:
core_weather["target"] = core_weather.shift(-1)["temp_max"]


In [ ]:
core_weather


In [ ]:
core_weather = core_weather.iloc[:-1, :].copy()


In [ ]:
core_weather


In [ ]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)


In [ ]:
predictors = ["precip", "temp_max", "temp_min"]


In [ ]:
train = core_weather.loc[:"2020-12-31"]
test = core_weather.loc["2021-01-01":]


In [ ]:
train


In [ ]:
test


In [ ]:
reg.fit(train[predictors], train["target"])


In [ ]:
predictions = reg.predict(test[predictors])


In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(test["target"], predictions)


In [ ]:
combined = pd.concat([test["target"], pd.Series(
    predictions, index=test.index)], axis=1)
combined.columns = ["actual", "predictions"]


In [ ]:
combined


In [ ]:
combined.plot()


In [ ]:
reg.coef_


In [ ]:
core_weather["month_max"] = core_weather["temp_max"].rolling(30).mean()

core_weather["month_day_max"] = core_weather["month_max"] / \
    core_weather["temp_max"]

core_weather["max_min"] = core_weather["temp_max"] / core_weather["temp_min"]


In [ ]:
core_weather = core_weather.iloc[30:, :].copy()


In [ ]:
def create_predictions(predictors, core_weather, reg):
    train = core_weather.loc[:"2020-12-31"]
    test = core_weather.loc["2021-01-01":]

    reg.fit(train[predictors], train["target"])
    predictions = reg.predict(test[predictors])

    error = mean_squared_error(test["target"], predictions)

    combined = pd.concat([test["target"], pd.Series(
        predictions, index=test.index)], axis=1)
    combined.columns = ["actual", "predictions"]
    return error, combined


In [ ]:
predictors = ["precip", "temp_max", "temp_min", "month_day_max", "max_min"]

error, combined = create_predictions(predictors, core_weather, reg)
error


In [ ]:
combined.plot()


In [ ]:
try :
    core_weather["monthly_avg"] = core_weather["temp_max"].groupby(
        core_weather.index.month,group_keys=False).apply(lambda x: x.expanding(1).mean())
    core_weather["day_of_year_avg"] = core_weather["temp_max"].groupby(
        core_weather.index.day_of_year,group_keys=False).apply(lambda x: x.expanding(1).mean())
except (Exception):
    core_weather["monthly_avg"] = core_weather["temp_max"].groupby(
        core_weather.index.month,group_keys=True).apply(lambda x: x.expanding(1).mean())
    core_weather["day_of_year_avg"] = core_weather["temp_max"].groupby(
        core_weather.index.day_of_year,group_keys=True).apply(lambda x: x.expanding(1).mean())

In [ ]:
error, combined = create_predictions(
    predictors + ["monthly_avg", "day_of_year_avg"], core_weather, reg)
error


In [ ]:
reg.coef_


In [ ]:
core_weather.corr()["target"]


In [ ]:
combined["diff"] = (combined["actual"] - combined["predictions"]).abs()


In [ ]:
combined.sort_values("diff", ascending=False).head(10)
